$\newcommand{\tensor}[1]{\boldsymbol{#1}}$

$\newcommand{\tensorthree}[1]{\mathbfcal{#1}}$
$\newcommand{\tensorfour}[1]{\mathbb{#1}}$

$\newcommand{\dar}{\, \text{d} a^r}$

$\newcommand{\dvr}{\, \text{d} v^r}$
$\newcommand{\dv}{\, \text{d} v}$

$\newcommand{\dt}{\, \text{d} t}$
$\newcommand{\dthe}{\, \text{d} \theta}$

$\newcommand{\tr}{\operatorname{tr}}$

## The nonlinear heat equation solver in FEniCS

In [1]:
from IPython.display import display_pretty, display_html, display_jpeg, display_png, display_json, display_latex, display_svg

from IPython.display import Image
Image(url='http://python.org/images/python-logo.gif')

### Functionals

The total **deformation gradient** can be can be decomposed as: 

\begin{equation}
    \tensor{F} = \tensor{F}_e ~ \tensor{F}_{\theta}
\end{equation}

The analysis restricted to isotropic materials, for which the thermal part of the deformation gradient is:

\begin{equation}
    \tensor{F}_{\theta} = \vartheta \left( \theta \right) ~ \tensor{I}
\end{equation}

The scalar $\theta = \upsilon \left( \theta \right)$ is the **thermal stretch ratio** in any material direction. In this case, the elastic and thermal Green strains become:

\begin{equation}
    \tensor{E}_{e} = \frac{1}{\vartheta^2} \left( \tensor{E} - \tensor{E}_{\theta} \right), \qquad \tensor{E}_{\theta} = \frac{1}{2} \left( \vartheta^2 - 1 \right) \tensor{I}
\end{equation}

The relationship holds:

\begin{equation}
    \tensor{I} + 2 \tensor{E} = \vartheta^2 \left( \tensor{I} + 2 \tensor{E}_{e} \right)
\end{equation}

Since the thermal stretch ratio $\vartheta$ and the coefficient of thermal expansion $\alpha$ are related by

\begin{equation}
    \alpha \left( \theta \right) = \frac{1}{\vartheta} \frac{\text{d} \vartheta}{\dthe} 
\end{equation}

the rate of elastic strain can be written as

\begin{equation}
    \dot{\tensor{E}}_{e} = \frac{1}{\vartheta^2 \left( \theta \right)} \left[ \dot{\tensor{E}} - \alpha \left( \theta \right) \left( \tensor{I} + 2 \tensor{E} \right) \dot{\theta} \right]
\end{equation}



### Stress Response

Within the model of the multiplicative decomposition, the Helmholtz free energy can be conveniently split into two
parts:

\begin{equation}
    {\varphi} \left( \tensor{u}, \theta \right) = {\varphi}_{e} \left( \tensor{E}_{e}, \theta \right) + {\varphi}_{\theta} \left( \theta \right)
\end{equation}

where ${\varphi}_{e}$ is an isotropic function of the elastic strain $\tensor{E}_{e}$ and the temperature $\theta$. This decomposition is physically appealing because the function ${\varphi}_{e}$ can be taken as one of the
well-known strain energy functions of the isothermal finite-strain elasticity, except that the coefficients of the strain-dependent terms are the functions of temperature, while the function ${\varphi}_{\theta}$ can be separately adjusted in accord with experimental data for the specific heat.

The time-rate of the free energy

\begin{equation}
    \dot{\varphi} = \frac{\text{d} \varphi \left( \tensor{u}, \theta \right)}{\dt} = \frac{\partial {\varphi}_{e}}{\partial \tensor{E}_{e}} : \dot{\tensor{E}_{e}} + \frac{\partial {\varphi}_{e}}{\partial \theta} ~ \dot{\theta} + \frac{\text{d} \varphi_{\theta}}{\dthe} ~ \dot{\theta}
\end{equation}

there follows

\begin{equation}
    \dot{\varphi} = \frac{1}{\vartheta^2} \frac{\partial {\varphi}_{e}}{\partial \tensor{E}_{e}} : \dot{\tensor{E}_{e}} - \left[ \frac{\alpha}{\vartheta^2} \frac{\partial {\varphi}_{e}}{\partial \tensor{E}_{e}}: \left( \tensor{I} + 2 \tensor{E} \right) - \frac{\partial {\varphi}_{e}}{\partial \theta} - \frac{\text{d} \varphi_{\theta}}{\dthe} \right] ~ \dot{\theta}
\end{equation}

The comparison with the energy equation:

\begin{equation}
    \dot{\varphi} = \frac{1}{\varrho^r} \tensor{S} : \dot{\tensor{E}} - \eta \dot{\theta}
\end{equation}

establishes the constitutive relations for the symmetric second Piola–Kirchhoff stress tensor $\tensor{S}$ and the specific entropy $\eta$. These
are:

\begin{equation}
    \tensor{S} = \frac{\varrho^r}{\vartheta^2} \frac{\partial {\varphi}_{e}}{\partial \tensor{E}_{e}}
\end{equation}

\begin{equation}
    \eta = \alpha \frac{\partial {\varphi}_{e}}{\partial \tensor{E}_{e}} : \left( \tensor{I} + 2 \tensor{E} \right) - \frac{\partial {\varphi}_{e}}{\partial \theta} - \frac{\text{d} \varphi_{\theta}}{\dthe}
\end{equation}

\begin{equation}
    \varrho^r = \upsilon^3 \varrho^{\theta}
\end{equation}

\begin{equation}
    \tensor{S} = \vartheta ~ \tensor{S}_{e}, \qquad \tensor{S}_{e} = \varrho^{\theta} \frac{\partial {\varphi}_{e}}{\partial \tensor{E}_{e}}
\end{equation}

\begin{equation}
    \varrho^{\theta} \frac{\partial {\varphi}_{e}}{\partial \tensor{E}_{e}} = \frac{1}{2} \lambda \left( \theta \right) \left( \tr{\tensor{E}_{e}} \right)^{2} + \mu \left( \theta \right) ~ \tensor{E}_{e} : \tensor{E}_{e}
\end{equation}

where $\lambda \left( \theta \right)$ and $\mu \left( \theta \right)$ are the temperature-dependent Lamé moduli. It follows that

\begin{equation}
    \tensor{S}_{e} = \tensorfour{C} \left( \tensor{u}, \theta \right) : \tensor{E}_{e}
\end{equation}

into $\tensor{S} = \vartheta \tensor{S}_{e}$, the stress response becomes

\begin{equation}
    \tensor{S} = \frac{1}{\vartheta} \left[ \lambda \left( \tr{\tensor{E}} \right) \tensor{I} + 2 \mu \tensor{E} \right] - \frac{3}{2} \left[ \vartheta - \frac{1}{\vartheta} \right] \kappa \tensor{I},
\end{equation}

where $\kappa$ refers to the temperature-dependent bulk modulus. This is an exact expression for the thermoelastic stress response in the case of quadratic representation of $\varphi_e$ e in terms of the finite elastic strain $\tensor{E}_{e}$. If the Lamé moduli are taken to be temperature-independent, and if the approximation $\upsilon \approx 1 + \alpha^{r} \left( \theta - \theta^{r} \right)$ is used ($\alpha^{r}$ o being the coefficient of linear thermal expansion at $\theta - \theta^{r}$), that reduces to

\begin{equation}
    \tensor{S} = \lambda^{r} \left( \tr{\tensor{E}} \right) \tensor{I} + 2 \mu^{r} \tensor{E} - 3 \alpha^{r} \left( \theta - \theta^{r} \right) \kappa^{r} \tensor{I}
\end{equation}

### Entropy expression

In the case of quadratic strain energy representation, there is a relationship $\varrho^r {\varphi}_{e} = \vartheta ^ 3 ~ \tensor{S}_{e} : \tensor{E}_{e}/2 $, so that

\begin{equation}
    \varrho^r \frac{\partial {\varphi}_{e}}{\partial \theta} = \frac{3}{2} \vartheta^2 \frac{\text{d} \varphi_{\theta}}{\dthe} \tensor{S}_{e} : \tensor{E}_{e} + \frac{1}{2} \vartheta^3 \frac{\partial \tensor{S}_{e}}{\partial \theta} : \tensor{E}_{e}
\end{equation}

\begin{equation}
    \varrho^r \frac{\partial {\varphi}_{e}}{\partial \theta} = \frac{3}{2} \alpha \left[ \tensor{S} : \tensor{E} - \frac{1}{2} \left( \vartheta^2 - 1 \right) ~ \tr{\tensor{S}} \right] + \frac{1}{2} \vartheta^3 \frac{\partial {\tensor{S}}_{e}}{\partial \theta} : \tensor{E}_{e}
\end{equation} 

The coefficient of thermal expansion $\alpha$ can be readily verified that

\begin{equation}
    \vartheta \frac{\partial {\tensor{S}}_{e}}{\partial \theta} = \frac{\partial {\tensor{S}}}{\partial \theta} + \alpha \left( \tensor{S} + 3 \vartheta \kappa \tensor{I} \right)
\end{equation}

\begin{equation}
    \vartheta^{3} \frac{\partial {\tensor{S}}_{e}}{\partial \theta} : \tensor{E}_{e} = \frac{\partial {\tensor{S}}}{\partial \theta} : \left[ \tensor{E} - \frac{\left( \vartheta^{2} - 1 \right)}{2} ~ \tensor{I} \right] + \alpha \left[ \tensor{S} : \tensor{E} + \frac{\left( 1 + \vartheta^{2} \right)}{2} \tr{\tensor{S}} \right]
\end{equation}

\begin{equation}
    \varrho^r \frac{\partial {\varphi}_{e}}{\partial \theta} = 2 \alpha ~ \tensor{S} : \tensor{E} + \frac{\alpha \left( 2 - \vartheta^{2} \right)}{2} \tr{\tensor{S}} + \frac{1}{2} \frac{\partial {\tensor{S}}}{\partial \theta} : \left[ \tensor{E} - \frac{\left( \vartheta^{2} - 1 \right)}{2} ~ \tensor{I} \right]
\end{equation}

\begin{equation}
    \eta = \frac{1}{2 \varrho^r} \left[ 3 \vartheta \alpha \kappa \tensor{I} - \frac{\partial {\tensor{S}}}{\partial \theta} \right] : \left[ \tensor{E} - \frac{\left( \vartheta^{2} - 1 \right)}{2} ~ \tensor{I} \right] - \frac{\text{d} \varphi_{\theta}}{\dthe}
\end{equation}
 
Recalling the standard expression for the latent heat $\tensor{\varepsilon}$, we finally have

\begin{equation}
    \eta = \frac{1}{2} \left( \frac{\tensor{\varepsilon}}{\theta} + \frac{\vartheta \alpha \kappa}{\varrho^r} \tensor{I} \right) : \left( \tensor{E} - \frac{\left( \vartheta^{2} - 1 \right)}{2} ~ \tensor{I} \right) - \frac{\text{d} \varphi_{\theta}}{\dthe}
\end{equation}

This is an exact expression for the entropy $\eta$ within the approximation used for the elastic strain energy function. The second-order tensor of the latent heat $\tensor{\varepsilon}$ can be calculated as

\begin{equation}
    \tensor{\varepsilon} = - \frac{\theta}{\varrho^r} \frac{\partial {\tensor{S}}}{\partial \theta} = - \frac{\theta}{\varrho^r} \left( \vartheta \frac{\partial {\tensor{S}}}{\partial \theta} -\alpha \left( \tensor{S} + 3 \vartheta \kappa \tensor{I} \right) \right)
\end{equation}

which gives

\begin{equation}
    \tensor{\varepsilon} =  \frac{\theta}{\varrho^r} \left( \alpha \left( \tensor{S} + 3 \vartheta \kappa \tensor{I} \right) - \frac{1}{\vartheta} \frac{\text{d} \tensorfour{C}}{\dthe} : \left( \tensor{E} - \frac{\left( \vartheta^{2} - 1 \right)}{2} ~ \tensor{I} \right) \right)
\end{equation}

If the elastic moduli are independent of the temperature, and if the stress components are much smaller than the elastic bulk modulus, then the specific heat becomes $\tensor{\varepsilon} = 3 \vartheta \alpha \theta \kappa \tensor{I} / {\varrho^r}$, while the entropy expression reduces to

\begin{equation}
    \eta = \frac{3}{\varrho^r} \vartheta \alpha \kappa \left( \tr{\tensor{E}} - \frac{3}{2} \left( \vartheta^2 - 1 \right) \right) - \frac{\text{d} \varphi_{\theta}}{\dthe}
\end{equation}

The function $\varphi_{\theta}$ can be selected according to experimental data for the specific heat $c_{E} = \theta \partial \eta / \partial \theta$. For example, if we take

\begin{equation}
    \varphi_{\theta} = -\frac{1}{2} \left( \frac{c_{E}}{\theta^r} + \frac{9 \left( \alpha^r \right)^2 \kappa^r}{\varrho^r} \right) \left( \theta - \theta^r \right)^2
\end{equation}

then becomes

\begin{equation}
    \eta = \frac{3}{\varrho^r} \alpha^r \kappa^r \tr{\tensor{E}} + \frac{c_{E}}{\theta^r} \left( \theta - \theta^r \right)
\end{equation}

which is in agreement with the classical result from the linearized theory of thermoelasticity. The approximations $\alpha \approx \alpha^r$ and $\vartheta \approx 1 + \alpha^r \left( \theta - \theta^r \right)$ are used in the above derivation.